In [1]:
source('utils.r')
source('selinf_functions.r')
source('metrics.r')
source('data_generator.r')
source('cov_matrix.r')

Warning message in file(filename, "r", encoding = encoding):
"cannot open file 'utils.r': No such file or directory"


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


## Simulating random time
In this example we simulate data from a certain number of subjects and assume that time (introduced as a dummy variable) has a random effect. The rest of the computation is the same as before.